In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
df = pd.read_pickle('df_1to1999_cleaned.pkl')

In [4]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA"
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ..."


In [5]:
import requests
from bs4 import BeautifulSoup
import re

In [108]:
def get_home_values(df):
    url = 'http://www.zillow.com/homes/for_sale/featured_sort'
    for row in xrange(1):
#     for row in xrange(df.shape[0]):
        lat, lon = df.ix[row, 'latitude'], df.ix[row, 'longitude']
        url = url+'/{0},{1},{2},{3}_rect/15_zm/'.format(lat+.003,lon+.003,lat-.003,lon+.003)
        r = requests.get(url)
#         soup = BeautifulSoup(r.content, 'html.parser')
        soup = BeautifulSoup(r.text, 'html.parser')
#         g_data = soup.find_all("div", {"class": "property-info"})
        g_data = soup.find_all("dt", {"class": "zestimate"})
        for item in g_data:
            print item.text
            
        
       

In [109]:
get_home_values(df)

####This gives me nothing.  Let's look at the code line by line.

In [110]:
url = 'http://www.zillow.com/homes/for_sale/featured_sort/47.5348578,-122.3601857,47.5288578,-122.3601857_rect/15_zm/'

In [111]:
r = requests.get(url)

In [112]:
soup = BeautifulSoup(r.text, 'html.parser')
# print soup.prettify()

In [117]:
g_data = soup.find_all("div", {"class": "property-info"}) #This gives me an empty list

In [118]:
g_data

[]

In [115]:
g_data = soup.find_all("dt", {"class": "zestimate"}) #This gives me an empty list

In [116]:
g_data

[]

In [119]:
g_data = soup.find_all("div", {"id": "container"}) #This gives me a list of one element that doesn't contain the property values
                                                    # I am looking for.

In [120]:
g_data[0]

<div class="search-content-container" id="container"><div class="active-view"><div class="property-data-column yui3-u " id="c-column"><div class="inner-property-data-column" id="inner-c-column"><div class="search-list-col" id="list-container-column"><div class="active-view"><div class="" data-zmm-page="Search-List" id="list-results-container"><div class="photoex hide" id="photoex-viewer"><div class="photoex-photos hide" id="photo-viewer"></div><div class="mapsViews hide" id="map-viewer"></div></div><!-- Property results --><div class="list-container " id="list-container"><div id="inner-list-container"><div class="nav-ad above-nav-ad" id="nav-ad-container"><div id="list-nav-ad-loading"><span class="zsg-loading-spinner"></span> Loading </div><div class="nav-ad-property-listing" id="nav-ad-gpt-container"><div class="google-ad-config" data-frame-id="gpt-ad-da914a56-c153-4844-90df-3a1ce01e2a0b" id="gpt-ad-da914a56-c153-4844-90df-3a1ce01e2a0b-config" style="display: none;">{"targetDiv":"gpt-

In [121]:
type(g_data)

bs4.element.ResultSet

In [123]:
g_data[0].find_all("dt", {"class": "zestimate"}) #Nothing

[]

####This is just not working.  BeautifulSoup is not working.  I need to try another approach to getting the socio-economic data.

In [17]:
get_home_values(df)

http://www.zillow.com/homes/for_sale/featured_sort/47.5348578,-122.3601857,47.5288578,-122.3601857_rect/15_zm/


In [124]:
r.text.find('$')

22005

In [125]:
[m.start() for m in re.finditer('test', 'test test test test')] #some Regex code to find re-occurences of textg

[0, 5, 10, 15]

In [128]:
positions = [m.start() for m in re.finditer('\$', r.text)] #I had to 'escape' the $

In [129]:
for pos in positions:
    print r.text[pos:pos+10]

$ui$TopNav
$TopNav201
$ui$TopNav
$TopNav201
$ui$TopNav
$TopNav201
$ui$TopNav
$TopNav201
$50,000+</
$75,000+</
$100,000+<
$150,000+<
$200,000+<
$250,000+<
$300,000+<
$400,000+<
$500,000+<
$500+</a><
$750+</a><
$1,000+</a
$1,250+</a
$1,500+</a
$1,750+</a
$2,000+</a
$2,500+</a
$3,000+</a
$100,000</
$200,000</
$300,000</
$400,000</
$500,000</
$600,000</
$700,000</
$800,000</
$900,000</
$1,000</a>
$1,500</a>
$2,000</a>
$2,500</a>
$3,000</a>
$3,500</a>
$4,000</a>
$4,500</a>
$5,000</a>
$</div><in
$</div><in
$</div><in
$</div><in
$<span cla
$links$Log
$LoginLink
$links$Reg
$RegisterL
$/g,"")},r


#### I don't get the property values I see when I go to "inspect element" at the web address.  Where are the property values?

#####socrata seattle.data.gov property values: Nothing here.

####Trying King County Dept of Assessments at http://info.kingcounty.gov/Assessor/eRealProperty/default.aspx